In [10]:
# Step 1: Install libraries
!pip install transformers gradio --quiet


In [11]:
# Step 2: Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr


In [12]:
# Step 3: Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


In [15]:
def chat(user_input, history):
    # If there's no history, create a blank tensor
    if history is None:
        history = []
        chat_history_ids = None
    else:
        chat_history_ids = history

    # Encode the input
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append to chat history (context)
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generate bot response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )

    # Decode bot response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response, chat_history_ids



In [16]:
gr.Interface(
    fn=chat,
    inputs=["text", "state"],
    outputs=["text", "state"],
    title="DialoGPT Chatbot",
    description="Talk to this AI chatbot! Built using HuggingFace & Gradio.",
    allow_flagging="never"
).launch(share=True)



/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bbb1dc901952d2b997.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
